# Todo Agent (TickTick MCP)

Use this notebook to chat with a todo-focused agent that has TickTick MCP tools loaded.

Prereqs:
- Start TickTick MCP: `docker compose --profile ticktick up -d ticktick-mcp`
- If this is the first run, do auth first: `docker compose --profile ticktick --profile auth run --rm ticktick-mcp-auth`
- Set `OPENAI_API_KEY` in `.env`
- Optional: set `TICKTICK_MCP_URL` (default `http://localhost:8002/mcp`)


In [ ]:
import os
from pathlib import Path
import nest_asyncio
from dotenv import load_dotenv

from autogen_ext.tools.mcp import mcp_server_tools, StreamableHttpServerParams
from autogen_agentchat.agents import AssistantAgent
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_agentchat.messages import TextMessage
from autogen_core import CancellationToken

nest_asyncio.apply()
load_dotenv(Path.cwd() / ".env")

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
TICKTICK_MCP_URL = os.getenv("TICKTICK_MCP_URL", "http://localhost:8002/mcp")
OPENAI_MODEL = os.getenv("OPENAI_MODEL", "gpt-5-2025-08-07")

if not OPENAI_API_KEY:
    raise RuntimeError("OPENAI_API_KEY is not set")

print("TickTick MCP URL:", TICKTICK_MCP_URL)
print("OpenAI model:", OPENAI_MODEL)


In [ ]:
prompt = """
You are the Todo Agent. Your job is to help me plan by using TickTick tasks.
Use the TickTick MCP tools to list, search, and update tasks as needed.
Be concise. Ask one clarifying question at a time when needed.
Confirm destructive actions before executing them.
""".strip()

params = StreamableHttpServerParams(url=TICKTICK_MCP_URL, timeout=10.0)
tools = await mcp_server_tools(params)

todo_agent = AssistantAgent(
    name="todo_agent",
    model_client=OpenAIChatCompletionClient(
        model=OPENAI_MODEL,
        api_key=OPENAI_API_KEY,
    ),
    system_message=prompt,
    tools=tools,
)

print(f"Loaded {len(tools)} TickTick tools")


In [ ]:
async def ask_todo_agent(text: str) -> str:
    message = TextMessage(content=text, source="user")
    response = await todo_agent.on_messages([message], CancellationToken())
    return response.chat_message.content

answer = await ask_todo_agent("What should I focus on today?")
print(answer)
